<a href="https://colab.research.google.com/github/fujitachi/kyodo/blob/master/100%E6%9C%AC%E3%83%8E%E3%83%83%E3%82%AF61_80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler


if 'PG_PORT' in os.environ:

    host = 'db'
    port = os.environ['PG_PORT']
    database = os.environ['PG_DATABASE']
    user = os.environ['PG_USER']
    password = os.environ['PG_PASSWORD']

    # pd.read_sql用のコネクタ
    conn = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('../data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/answer')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }

    df_customer = pd.read_csv("../data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("../data/category.csv", dtype=dtype)
    df_product = pd.read_csv("../data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("../data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("../data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("../data/geocode.csv", dtype=dtype)


Cloning into '100knocks-preprocess'...
remote: Enumerating objects: 1959, done.
remote: Counting objects: 100% (754/754), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 1959 (delta 619), reused 637 (delta 559), pack-reused 1205
Receiving objects: 100% (1959/1959), 21.00 MiB | 19.85 MiB/s, done.
Resolving deltas: 100% (1100/1100), done.


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
print(df_receipt.head(10))


   sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181118   1542499200   S13008        1132               2   
2   20170712   1499817600   S14028        1102               1   
3   20190205   1549324800   S14042        1132               1   
4   20180821   1534809600   S14025        1102               2   
5   20190605   1559692800   S13003        1112               1   
6   20181205   1543968000   S14024        1102               2   
7   20190922   1569110400   S14040        1102               1   
8   20170504   1493856000   S13020        1112               2   
9   20191010   1570665600   S14027        1102               1   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81  
2  CS028414000014  P060101005         1     170  
3  ZZ000000000000  P050301001         1      25  
4  CS025415000050  P060

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [ ]:
# 指定された列を含むデータフレームの作成
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
df_selected = df_receipt[selected_columns]

print(df_selected.head(10))

   sales_ymd     customer_id  product_cd  amount
0   20181103  CS006214000001  P070305012     158
1   20181118  CS008415000097  P070701017      81
2   20170712  CS028414000014  P060101005     170
3   20190205  ZZ000000000000  P050301001      25
4   20180821  CS025415000050  P060102007      90
5   20190605  CS003515000195  P050102002     138
6   20181205  CS024514000042  P080101005      30
7   20190922  CS040415000178  P070501004     128
8   20170504  ZZ000000000000  P071302010     770
9   20191010  CS027514000015  P071101003     680


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [ ]:
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
df_selected = df_receipt[selected_columns]

df_selected = df_selected.rename(columns={'sales_ymd': 'sales_date'})

print(df_selected.head(10))

   sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680


---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
condition = df_receipt['customer_id'] == 'CS018205000001'
filtered_data = df_receipt.loc[condition, selected_columns]

print(filtered_data)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
condition = (df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000)
filtered_data = df_receipt.loc[condition, selected_columns]

print(filtered_data)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']
condition = ((df_receipt['customer_id'] == 'CS018205000001') &
             ((df_receipt['amount'] >= 1000) | (df_receipt['quantity'] >= 5)))
filtered_data = df_receipt.loc[condition, selected_columns]

print(filtered_data)

       sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
condition = (df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['amount'] >= 1000) & (df_receipt['amount'] <= 2000)
filtered_data = df_receipt.loc[condition, selected_columns]

print(filtered_data)

       sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
selected_columns = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
condition = (df_receipt['customer_id'] == 'CS018205000001') & (df_receipt['product_cd'] != 'P071401019')
filtered_data = df_receipt.loc[condition, selected_columns]

print(filtered_data)

       sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
>
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('not(prefecture_cd == "13") & not(floor_area > 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [ ]:
filtered_data = df_store[df_store['store_cd'].str.startswith('S14')]

print(filtered_data.head(10))

   store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [ ]:
filtered_data = df_customer[df_customer['customer_id'].str.endswith('1')]

print(filtered_data.head(10))

        customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
1    CS037613000071         六角 雅彦         9     不明  1952-04-01   66  136-0076   
3    CS028811000001        堀井 かおり         1     女性  1933-03-27   86  245-0016   
14   CS040412000191         川井 郁恵         1     女性  1977-01-05   42  226-0021   
31   CS028314000011        小菅 あおい         1     女性  1983-11-26   35  246-0038   
56   CS039212000051        藤島 恵梨香         1     女性  1997-02-03   22  166-0001   
59   CS015412000111         松居 奈月         1     女性  1972-10-04   46  136-0071   
63   CS004702000041          野島 洋         0     男性  1943-08-24   75  176-0022   
74   CS041515000001         栗田 千夏         1     女性  1967-01-02   52  206-0001   
85   CS029313000221        北条 ひかり         1     女性  1987-06-19   31  279-0011   
102  CS034312000071         望月 奈央         1     女性  1980-09-20   38  213-0026   

                     address application_store_cd  application_date  \
1         東京都江東区南砂**********         

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [ ]:
filtered_data = df_store[df_store['address'].str.contains('横浜市')]

print(filtered_data)

   store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   
52   S14006       葛が谷店            14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  3

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [ ]:
filtered_data = df_customer[df_customer['status_cd'].str.match(r'^[A-F]')]

print(filtered_data.head(10))

       customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子         1     女性  1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂         1     女性  1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希         9     不明  1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子         1     女性  1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆         1     女性  1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太         0     男性  1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美         1     女性  1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒         1     女性  1977-06-27   41  157-0067   

                     address application_store_cd  application_date  \
2        東京都渋谷区代々木**********               S1303

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
filtered_data = df_customer[df_customer['status_cd'].str.match(r'.*[1-9]$')]

print(filtered_data.head(10))

       customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
4   CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   
9   CS033513000180          安斎 遥         1     女性  1962-07-11   56  241-0823   
12  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
14  CS040412000191         川井 郁恵         1     女性  1977-01-05   42  226-0021   
16  CS009315000023         皆川 文世         1     女性  1980-04-15   38  154-0012   
22  CS015315000033        福士 璃奈子         1     女性  1983-03-17   36  135-0043   
23  CS023513000066         神戸 そら         1     女性  1961-12-17   57  210-0005   
24  CS035513000134         市川 美帆         1     女性  1960-03-27   59  156-0053   
27  CS001515000263         高松 夏空         1     女性  1962-11-09   56  144-0051   
28  CS040314000027       鶴田 きみまろ         9     不明  1986-03-26   33  226-0027   

                     address application_store_cd  application_date  \
4        東京都大田区仲六郷**********               S1300

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
filtered_data = df_customer[df_customer['status_cd'].str.match(r'^[A-F].*[1-9]$')]

print(filtered_data.head(10))

        customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
12   CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
68   CS022513000105        島村 貴美子         1     女性  1962-03-12   57  249-0002   
71   CS001515000096         水野 陽子         9     不明  1960-11-29   58  144-0053   
122  CS013615000053         西脇 季衣         1     女性  1953-10-18   65  261-0026   
144  CS020412000161          小宮 薫         1     女性  1974-05-21   44  174-0042   
178  CS001215000097        竹中 あさみ         1     女性  1990-07-25   28  146-0095   
252  CS035212000007        内村 恵梨香         1     女性  1990-12-04   28  152-0023   
259  CS002515000386         野田 コウ         1     女性  1963-05-30   55  185-0013   
293  CS001615000372        稲垣 寿々花         1     女性  1956-10-29   62  144-0035   
297  CS032512000121         松井 知世         1     女性  1962-09-04   56  210-0011   

                      address application_store_cd  application_date  \
12   神奈川県横浜市港北区日吉本町**********       

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
filtered_data = df_store[df_store['tel_no'].str.match(r'\d{3}-\d{3}-\d{4}')]

print(filtered_data)

   store_cd store_name prefecture_cd prefecture              address  \
0    S12014       千草台店            12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1    S13002       国分寺店            13        東京都         東京都国分寺市本多二丁目   
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
11   S13052        森野店            13        東京都          東京都町田市森野三丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
sorted_data = df_customer.sort_values(by='birth_day')
print(sorted_data.head(10))

          customer_id customer_name gender_cd gender   birth_day  age  \
18817  CS003813000014        村山 菜々美         1     女性  1928-11-26   90   
12328  CS026813000004         吉村 朝陽         1     女性  1928-12-14   90   
15682  CS018811000003         熊沢 美里         1     女性  1929-01-07   90   
15302  CS027803000004         内村 拓郎         0     男性  1929-01-12   90   
1681   CS013801000003         天野 拓郎         0     男性  1929-01-15   90   
7511   CS001814000022         鶴田 里穂         1     女性  1929-01-28   90   
2378   CS016815000002         山元 美紀         1     女性  1929-02-22   90   
4680   CS009815000003         中田 里穂         1     女性  1929-04-08   89   
16070  CS005813000015        金谷 恵梨香         1     女性  1929-04-09   89   
6305   CS012813000013         宇野 南朋         1     女性  1929-04-09   89   

      postal_cd                 address application_store_cd  \
18817  182-0007     東京都調布市菊野台**********               S13003   
12328  251-0043   神奈川県藤沢市辻堂元町**********               S14026   
1568

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
sorted_data = df_customer.sort_values(by='birth_day', ascending=True)
print(sorted_data.head(10))

          customer_id customer_name gender_cd gender   birth_day  age  \
18817  CS003813000014        村山 菜々美         1     女性  1928-11-26   90   
12328  CS026813000004         吉村 朝陽         1     女性  1928-12-14   90   
15682  CS018811000003         熊沢 美里         1     女性  1929-01-07   90   
15302  CS027803000004         内村 拓郎         0     男性  1929-01-12   90   
1681   CS013801000003         天野 拓郎         0     男性  1929-01-15   90   
7511   CS001814000022         鶴田 里穂         1     女性  1929-01-28   90   
2378   CS016815000002         山元 美紀         1     女性  1929-02-22   90   
4680   CS009815000003         中田 里穂         1     女性  1929-04-08   89   
16070  CS005813000015        金谷 恵梨香         1     女性  1929-04-09   89   
6305   CS012813000013         宇野 南朋         1     女性  1929-04-09   89   

      postal_cd                 address application_store_cd  \
18817  182-0007     東京都調布市菊野台**********               S13003   
12328  251-0043   神奈川県藤沢市辻堂元町**********               S14026   
1568

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_receipt['rank'] = df_receipt['amount'].rank(method='min', ascending=False)

top_10 = df_receipt[['customer_id', 'amount', 'rank']].sort_values(by='rank').head(10)
print(top_10)

          customer_id  amount  rank
1202   CS011415000006   10925   1.0
62317  ZZ000000000000    6800   2.0
54095  CS028605000002    5780   3.0
4632   CS015515000034    5480   4.0
72747  ZZ000000000000    5480   4.0
10320  ZZ000000000000    5480   4.0
97294  CS021515000089    5440   7.0
28304  ZZ000000000000    5440   7.0
92246  CS009415000038    5280   9.0
68553  CS040415000200    5280   9.0


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_receipt['rank'] = df_receipt['amount'].rank(method='dense', ascending=False)

top_10 = df_receipt[['customer_id', 'amount', 'rank']].sort_values(by='rank').head(10)
print(top_10)

          customer_id  amount  rank
1202   CS011415000006   10925   1.0
62317  ZZ000000000000    6800   2.0
54095  CS028605000002    5780   3.0
4632   CS015515000034    5480   4.0
72747  ZZ000000000000    5480   4.0
10320  ZZ000000000000    5480   4.0
97294  CS021515000089    5440   5.0
28304  ZZ000000000000    5440   5.0
92246  CS009415000038    5280   6.0
68553  CS040415000200    5280   6.0


---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [ ]:
row_count = len(df_receipt)

print("レシート明細データの件数:", row_count)

レシート明細データの件数: 104681


---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
u_count = df_receipt['customer_id'].nunique()

print("ユニークな顧客IDの件数:", u_count)

ユニークな顧客IDの件数: 8307


---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
result = df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).reset_index()

print(result)

   store_cd  amount  quantity
0    S12007  638761      2099
1    S12013  787513      2425
2    S12014  725167      2358
3    S12029  794741      2555
4    S12030  684402      2403
5    S13001  811936      2347
6    S13002  727821      2340
7    S13003  764294      2197
8    S13004  779373      2390
9    S13005  629876      2004
10   S13008  809288      2491
11   S13009  808870      2486
12   S13015  780873      2248
13   S13016  793773      2432
14   S13017  748221      2376
15   S13018  790535      2562
16   S13019  827833      2541
17   S13020  796383      2383
18   S13031  705968      2336
19   S13032  790501      2491
20   S13035  715869      2219
21   S13037  693087      2344
22   S13038  708884      2337
23   S13039  611888      1981
24   S13041  728266      2233
25   S13043  587895      1881
26   S13044  520764      1729
27   S13051  107452       354
28   S13052  100314       250
29   S14006  712839      2284
30   S14010  790361      2290
31   S14011  805724      2434
32   S1401

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
l_customer = df_receipt.groupby('customer_id')['sales_ymd'].max().reset_index()

print(l_customer.head(10))

      customer_id  sales_ymd
0  CS001113000004   20190308
1  CS001114000005   20190731
2  CS001115000010   20190405
3  CS001205000004   20190625
4  CS001205000006   20190224
5  CS001211000025   20190322
6  CS001212000027   20170127
7  CS001212000031   20180906
8  CS001212000046   20170811
9  CS001212000070   20191018


---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
o_customer = df_receipt.groupby('customer_id')['sales_ymd'].min().reset_index()

print(o_customer.head(10))

      customer_id  sales_ymd
0  CS001113000004   20190308
1  CS001114000005   20180503
2  CS001115000010   20171228
3  CS001205000004   20170914
4  CS001205000006   20180207
5  CS001211000025   20190322
6  CS001212000027   20170127
7  CS001212000031   20180906
8  CS001212000046   20170811
9  CS001212000070   20191018


---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
s_customer = df_receipt.groupby('customer_id')['sales_ymd'].agg(['min', 'max']).reset_index()

dates = s_customer[s_customer['min'] != s_customer['max']]

print(dates.head(10))

       customer_id       min       max
1   CS001114000005  20180503  20190731
2   CS001115000010  20171228  20190405
3   CS001205000004  20170914  20190625
4   CS001205000006  20180207  20190224
13  CS001214000009  20170306  20190902
14  CS001214000017  20180828  20191006
16  CS001214000048  20171109  20190929
17  CS001214000052  20180208  20190617
20  CS001215000005  20170206  20181021
21  CS001215000040  20170214  20171022


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
a_store = df_receipt.groupby('store_cd')['amount'].mean().reset_index()

top5 = a_store.sort_values(by='amount', ascending=False).head(5)

print(top5)

   store_cd      amount
28   S13052  402.867470
12   S13015  351.111960
7    S13003  350.915519
30   S14010  348.791262
5    S13001  348.470386


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
m_store = df_receipt.groupby('store_cd')['amount'].median().reset_index()

top5 = m_store.sort_values(by='amount', ascending=False).head(5)

print(top5)

   store_cd  amount
28   S13052   190.0
30   S14010   188.0
51   S14050   185.0
44   S14040   180.0
7    S13003   180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [ ]:
m_store = df_receipt.groupby('store_cd')['product_cd'].agg(lambda x: x.mode()[0]).reset_index()

print(m_store.head(10))

  store_cd  product_cd
0   S12007  P060303001
1   S12013  P060303001
2   S12014  P060303001
3   S12029  P060303001
4   S12030  P060303001
5   S13001  P060303001
6   S13002  P060303001
7   S13003  P071401001
8   S13004  P060303001
9   S13005  P040503001


---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [ ]:
v_store = df_receipt.groupby('store_cd')['amount'].var().reset_index()

top5 = v_store.sort_values(by='amount', ascending=False).head(5)

print(top5)

   store_cd         amount
28   S13052  441863.252526
31   S14011  306442.242432
42   S14034  297068.392740
5    S13001  295558.842618
12   S13015  295427.197086


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [ ]:
std_store = df_receipt.groupby('store_cd')['amount'].std().reset_index()

top5_stores = std_store.sort_values(by='amount', ascending=False).head(5)

print(top5_stores)

   store_cd      amount
28   S13052  664.727954
31   S14011  553.572256
42   S14034  545.039808
5    S13001  543.653237
12   S13015  543.532149


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
percentiles = [0.25, 0.50, 0.75, 1.0]
amount_percentiles = df_receipt['amount'].quantile(percentiles)

print(amount_percentiles)

0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64


---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
average_store = df_receipt.groupby('store_cd')['amount'].mean()

high_average_amount = average_store[average_store >= 330]

print(high_average_amount)

store_cd
S12013    330.194130
S13001    348.470386
S13003    350.915519
S13004    330.943949
S13015    351.111960
S13019    330.208616
S13020    337.879932
S13052    402.867470
S14010    348.791262
S14011    335.718333
S14026    332.340588
S14045    330.082073
S14047    330.077073
Name: amount, dtype: float64


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
valid_customers_df = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

total_amount_per_customer = valid_customers_df.groupby('customer_id')['amount'].sum()

average_customer = total_amount_per_customer.mean()

print("全顧客の平均売上金額:", average_customer)

全顧客の平均売上金額: 2547.742234529256


---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
valid_customers_df = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

total_amount_per_customer = valid_customers_df.groupby('customer_id')['amount'].sum()

average_amount_per_customer = total_amount_per_customer.mean()

above_average_customers = total_amount_per_customer[total_amount_per_customer >= average_amount_per_customer]

print(above_average_customers.head(10))

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
CS001215000040    3496
CS001304000006    3726
CS001305000005    3485
CS001305000011    4370
CS001315000180    3300
Name: amount, dtype: int64


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [ ]:
merged_data = pd.merge(df_receipt, df_store[['store_cd', 'store_name']], on='store_cd', how='inner')

print(merged_data.head(10))

   sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1541203200   S14006         112               1   
1   20181116   1542326400   S14006         112               2   
2   20170118   1484697600   S14006        1162               1   
3   20190524   1558656000   S14006        1192               1   
4   20190419   1555632000   S14006         112               2   
5   20181119   1542585600   S14006        1152               2   
6   20171211   1512950400   S14006        1132               2   
7   20191021   1571616000   S14006        1112               2   
8   20170710   1499644800   S14006        1132               2   
9   20190805   1564963200   S14006         112               1   

      customer_id  product_cd  quantity  amount store_name  
0  CS006214000001  P070305012         1     158       葛が谷店  
1  ZZ000000000000  P080401001         1      48       葛が谷店  
2  CS006815000006  P050406035         1     220       葛が谷店  
3  CS006514000034  P060104003

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [ ]:
merged_data = pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], on='category_small_cd', how='inner')

print(merged_data.head(10))

   product_cd category_major_cd category_medium_cd category_small_cd  \
0  P040101001                04               0401            040101   
1  P040101002                04               0401            040101   
2  P040101003                04               0401            040101   
3  P040101004                04               0401            040101   
4  P040101005                04               0401            040101   
5  P040101006                04               0401            040101   
6  P040101007                04               0401            040101   
7  P040101008                04               0401            040101   
8  P040101009                04               0401            040101   
9  P040101010                04               0401            040101   

   unit_price  unit_cost category_small_name  
0       198.0      149.0                 弁当類  
1       218.0      164.0                 弁当類  
2       230.0      173.0                 弁当類  
3       248.0      

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
filtered_customers = df_customer[(df_customer['gender_cd'] == '1') & (~df_customer['customer_id'].str.startswith('Z'))]

customer_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

merged_data = pd.merge(filtered_customers, customer_sales, on='customer_id', how='left').fillna(0)

print(merged_data.head(10))

      customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性  1981-04-29   37  259-1113   
1  CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
2  CS028811000001        堀井 かおり         1     女性  1933-03-27   86  245-0016   
3  CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   
4  CS015414000103         奥野 陽子         1     女性  1977-08-09   41  136-0073   
5  CS033513000180          安斎 遥         1     女性  1962-07-11   56  241-0823   
6  CS035614000014        板倉 菜々美         1     女性  1954-07-16   64  154-0015   
7  CS011215000048         芦田 沙耶         1     女性  1992-02-01   27  223-0062   
8  CS009413000079         市川 コウ         1     女性  1975-12-28   43  158-0093   
9  CS040412000191         川井 郁恵         1     女性  1977-01-05   42  226-0021   

                    address application_store_cd  application_date  \
0      神奈川県伊勢原市粟窪**********               S14021          20

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
sales_by_date = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['sales_ymd'].nunique().reset_index()
top20_sales_by_date = sales_by_date.nlargest(20, 'sales_ymd')

sales_by_amount = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().reset_index()
top20_sales_by_amount = sales_by_amount.nlargest(20, 'amount')

merged_data = pd.merge(top20_sales_by_date, top20_sales_by_amount, on='customer_id', how='outer')

print(merged_data)

       customer_id  sales_ymd   amount
0   CS040214000008       23.0      NaN
1   CS010214000010       22.0  18585.0
2   CS015415000185       22.0  20153.0
3   CS010214000002       21.0      NaN
4   CS028415000007       21.0  19127.0
5   CS016415000141       20.0  18372.0
6   CS017415000097       20.0  23086.0
7   CS014214000023       19.0      NaN
8   CS021514000045       19.0      NaN
9   CS021515000172       19.0      NaN
10  CS022515000226       19.0      NaN
11  CS031414000051       19.0  19202.0
12  CS039414000052       19.0      NaN
13  CS007515000107       18.0      NaN
14  CS014415000077       18.0      NaN
15  CS021515000056       18.0      NaN
16  CS021515000211       18.0      NaN
17  CS022515000028       18.0      NaN
18  CS030214000008       18.0      NaN
19  CS031414000073       18.0      NaN
20  CS001605000009        NaN  18925.0
21  CS006515000023        NaN  18372.0
22  CS011414000106        NaN  18338.0
23  CS038415000104        NaN  17847.0
24  CS035414000024       

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [ ]:
num_rows_store = len(df_store)
num_rows_product = len(df_product)

total_combinations = num_rows_store * num_rows_product

print("店舗データと商品データの直積の件数:", total_combinations)

店舗データと商品データの直積の件数: 531590


---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [ ]:
df_daily_sales = df_receipt.groupby('sales_ymd')['amount'].sum().reset_index()

df_daily_sales['amount_diff'] = df_daily_sales['amount'].diff()

print(df_daily_sales.head(10))

   sales_ymd  amount  amount_diff
0   20170101   33723          NaN
1   20170102   24165      -9558.0
2   20170103   27503       3338.0
3   20170104   36165       8662.0
4   20170105   37830       1665.0
5   20170106   32387      -5443.0
6   20170107   23415      -8972.0
7   20170108   24737       1322.0
8   20170109   26718       1981.0
9   20170110   20143      -6575.0


---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [ ]:
df_daily_sales['sales_ymd'] = pd.to_datetime(df_daily_sales['sales_ymd'], format='%Y%m%d')

df_daily_sales['amount_lag1'] = df_daily_sales['amount'].shift(1)
df_daily_sales['amount_lag2'] = df_daily_sales['amount'].shift(2)
df_daily_sales['amount_lag3'] = df_daily_sales['amount'].shift(3)

print(df_daily_sales.head(10))

   sales_ymd  amount  amount_diff  amount_lag1  amount_lag2  amount_lag3
0 2017-01-01   33723          NaN          NaN          NaN          NaN
1 2017-01-02   24165      -9558.0      33723.0          NaN          NaN
2 2017-01-03   27503       3338.0      24165.0      33723.0          NaN
3 2017-01-04   36165       8662.0      27503.0      24165.0      33723.0
4 2017-01-05   37830       1665.0      36165.0      27503.0      24165.0
5 2017-01-06   32387      -5443.0      37830.0      36165.0      27503.0
6 2017-01-07   23415      -8972.0      32387.0      37830.0      36165.0
7 2017-01-08   24737       1322.0      23415.0      32387.0      37830.0
8 2017-01-09   26718       1981.0      24737.0      23415.0      32387.0
9 2017-01-10   20143      -6575.0      26718.0      24737.0      23415.0


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_merged = pd.merge(df_receipt, df_customer, on='customer_id')

df_merged['age_group'] = (df_merged['age'] // 10) * 10

summary = df_merged.pivot_table(
    values='amount',
    index='age_group',
    columns='gender_cd',
    aggfunc='sum',
    fill_value=0
).reset_index()

summary.columns = ['age_group', 'male_sales', 'female_sales', 'unknown_sales']

print(summary)

   age_group  male_sales  female_sales  unknown_sales
0         10        1591        149836           4317
1         20       72940       1363724          44328
2         30      177322        693047          50441
3         40       19355       9320791         483512
4         50       54320       6685192         342923
5         60      272469        987741          71418
6         70       13435         29764           2427
7         80       46360        262923           5111
8         90           0          6260              0


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [ ]:
df_sales_summary = summary

df_male = df_sales_summary[['age_group', 'male_sales']].copy()
df_female = df_sales_summary[['age_group', 'female_sales']].copy()
df_unknown = df_sales_summary[['age_group', 'unknown_sales']].copy()

df_male['gender_cd'] = '00'
df_female['gender_cd'] = '01'
df_unknown['gender_cd'] = '99'

df_sales_long = pd.concat([df_male.rename(columns={'male_sales': 'amount'}),
                           df_female.rename(columns={'female_sales': 'amount'}),
                           df_unknown.rename(columns={'unknown_sales': 'amount'})],
                          ignore_index=True)

print(df_sales_long)

    age_group   amount gender_cd
0          10     1591        00
1          20    72940        00
2          30   177322        00
3          40    19355        00
4          50    54320        00
5          60   272469        00
6          70    13435        00
7          80    46360        00
8          90        0        00
9          10   149836        01
10         20  1363724        01
11         30   693047        01
12         40  9320791        01
13         50  6685192        01
14         60   987741        01
15         70    29764        01
16         80   262923        01
17         90     6260        01
18         10     4317        99
19         20    44328        99
20         30    50441        99
21         40   483512        99
22         50   342923        99
23         60    71418        99
24         70     2427        99
25         80     5111        99
26         90        0        99


---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'], format='%Y%m%d')

print(df_receipt[['sales_ymd', 'receipt_no', 'receipt_sub_no']].head(10))

   sales_ymd  receipt_no  receipt_sub_no
0 2018-11-03         112               1
1 2018-11-18        1132               2
2 2017-07-12        1102               1
3 2019-02-05        1132               1
4 2018-08-21        1102               2
5 2019-06-05        1112               1
6 2018-12-05        1102               2
7 2019-09-22        1102               1
8 2017-05-04        1112               2
9 2019-10-10        1102               1


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
def unix_seconds_to_datetime(seconds):
    return pd.to_datetime(seconds, unit='s')

df_receipt['sales_datetime'] = df_receipt['sales_epoch'].apply(unix_seconds_to_datetime)

print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_datetime']].head(10))

   receipt_no  receipt_sub_no sales_datetime
0         112               1     2018-11-03
1        1132               2     2018-11-18
2        1102               1     2017-07-12
3        1132               1     2019-02-05
4        1102               2     2018-08-21
5        1112               1     2019-06-05
6        1102               2     2018-12-05
7        1102               1     2019-09-22
8        1112               2     2017-05-04
9        1102               1     2019-10-10


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
def extract_year_from_unix_seconds(seconds):
    return pd.to_datetime(seconds, unit='s').year

df_receipt['sales_year'] = df_receipt['sales_epoch'].apply(extract_year_from_unix_seconds)

print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_year']].head(10))

   receipt_no  receipt_sub_no  sales_year
0         112               1        2018
1        1132               2        2018
2        1102               1        2017
3        1132               1        2019
4        1102               2        2018
5        1112               1        2019
6        1102               2        2018
7        1102               1        2019
8        1112               2        2017
9        1102               1        2019


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [ ]:
def extract_month_from_unix_seconds(seconds):
    return pd.to_datetime(seconds, unit='s').strftime('%m')

df_receipt['sales_month'] = df_receipt['sales_epoch'].apply(extract_month_from_unix_seconds)

print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_month']].head(10))

   receipt_no  receipt_sub_no sales_month
0         112               1          11
1        1132               2          11
2        1102               1          07
3        1132               1          02
4        1102               2          08
5        1112               1          06
6        1102               2          12
7        1102               1          09
8        1112               2          05
9        1102               1          10


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [ ]:
def extract_day_from_unix_seconds(seconds):
    return pd.to_datetime(seconds, unit='s').strftime('%d')

df_receipt['sales_day'] = df_receipt['sales_epoch'].apply(extract_day_from_unix_seconds)

print(df_receipt[['receipt_no', 'receipt_sub_no', 'sales_day']].head(10))

   receipt_no  receipt_sub_no sales_day
0         112               1        03
1        1132               2        18
2        1102               1        12
3        1132               1        05
4        1102               2        21
5        1112               1        05
6        1102               2        05
7        1102               1        22
8        1112               2        04
9        1102               1        10


---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

df_customer_sales = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

df_customer_sales['binary_sales'] = df_customer_sales['amount'].apply(lambda x: 0 if x <= 2000 else 1)

print(df_customer_sales[['customer_id', 'amount', 'binary_sales']].head(10))

      customer_id  amount  binary_sales
0  CS001113000004    1298             0
1  CS001114000005     626             0
2  CS001115000010    3044             1
3  CS001205000004    1988             0
4  CS001205000006    3337             1
5  CS001211000025     456             0
6  CS001212000027     448             0
7  CS001212000031     296             0
8  CS001212000046     228             0
9  CS001212000070     456             0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [ ]:
def binarize_postal_cd(postal_cd):
    if 100 <= int(postal_cd[:3]) <= 209:
        return 1
    else:
        return 0

df_customer['is_tokyo'] = df_customer['postal_cd'].astype(str).apply(binarize_postal_cd)

df_receipt['has_sales'] = 1

df_customer_receipt = pd.merge(df_customer, df_receipt[['customer_id', 'has_sales']], on='customer_id', how='inner')

customer_count_by_bin = df_customer_receipt.groupby('is_tokyo')['has_sales'].sum()

print(customer_count_by_bin)

is_tokyo
0    34438
1    31244
Name: has_sales, dtype: int64


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [ ]:
def assign_prefecture_code(address):
    if '埼玉県' in address:
        return 11
    elif '千葉県' in address:
        return 12
    elif '東京都' in address:
        return 13
    elif '神奈川県' in address:
        return 14
    else:
        return None

df_customer['prefecture_code'] = df_customer['address'].apply(assign_prefecture_code)

print(df_customer[['customer_id', 'address', 'prefecture_code']].head(10))

      customer_id                 address  prefecture_code
0  CS021313000114    神奈川県伊勢原市粟窪**********               14
1  CS037613000071      東京都江東区南砂**********               13
2  CS031415000172     東京都渋谷区代々木**********               13
3  CS028811000001  神奈川県横浜市泉区和泉町**********               14
4  CS001215000145     東京都大田区仲六郷**********               13
5  CS020401000016      東京都板橋区若木**********               13
6  CS015414000103      東京都江東区北砂**********               13
7  CS029403000008      千葉県浦安市海楽**********               12
8  CS015804000004      東京都江東区北砂**********               13
9  CS033513000180  神奈川県横浜市旭区善部町**********               14


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [ ]:
df_customer_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

sales_quantiles = np.quantile(df_customer_sales['amount'], [0, 0.25, 0.5, 0.75, 1])

def assign_category(amount):
    if amount < sales_quantiles[1]:
        return 1
    elif amount < sales_quantiles[2]:
        return 2
    elif amount < sales_quantiles[3]:
        return 3
    else:
        return 4

df_customer_sales['category'] = df_customer_sales['amount'].apply(assign_category)

print(df_customer_sales[['customer_id', 'amount', 'category']].head(10))

      customer_id  amount  category
0  CS001113000004    1298         2
1  CS001114000005     626         2
2  CS001115000010    3044         3
3  CS001205000004    1988         3
4  CS001205000006    3337         3
5  CS001211000025     456         1
6  CS001212000027     448         1
7  CS001212000031     296         1
8  CS001212000046     228         1
9  CS001212000070     456         1


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [ ]:
def calculate_age_group(age):
    if age >= 60:
        return '60歳代'
    else:
        return f'{(age // 10) * 10}代'

df_customer['age_group'] = df_customer['age'].apply(calculate_age_group)

print(df_customer[['customer_id', 'birth_day', 'age_group']].head(10))

      customer_id   birth_day age_group
0  CS021313000114  1981-04-29       30代
1  CS037613000071  1952-04-01      60歳代
2  CS031415000172  1976-10-04       40代
3  CS028811000001  1933-03-27      60歳代
4  CS001215000145  1995-03-29       20代
5  CS020401000016  1974-09-15       40代
6  CS015414000103  1977-08-09       40代
7  CS029403000008  1973-08-17       40代
8  CS015804000004  1931-05-02      60歳代
9  CS033513000180  1962-07-11       50代


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [ ]:
df_customer['gender_age_category'] = df_customer['gender_cd'].astype(str) + '_' + df_customer['age_group']

print(df_customer[['customer_id', 'gender_cd', 'age_group', 'gender_age_category']].head(10))

      customer_id gender_cd age_group gender_age_category
0  CS021313000114         1       30代               1_30代
1  CS037613000071         9      60歳代              9_60歳代
2  CS031415000172         1       40代               1_40代
3  CS028811000001         1      60歳代              1_60歳代
4  CS001215000145         1       20代               1_20代
5  CS020401000016         0       40代               0_40代
6  CS015414000103         1       40代               1_40代
7  CS029403000008         0       40代               0_40代
8  CS015804000004         0      60歳代              0_60歳代
9  CS033513000180         1       50代               1_50代


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
df_gender_dummies = pd.get_dummies(df_customer['gender_cd'], prefix='gender')

df_customer_with_dummies = pd.concat([df_customer[['customer_id']], df_gender_dummies], axis=1)

print(df_customer_with_dummies.head(10))

      customer_id  gender_0  gender_1  gender_9
0  CS021313000114     False      True     False
1  CS037613000071     False     False      True
2  CS031415000172     False      True     False
3  CS028811000001     False      True     False
4  CS001215000145     False      True     False
5  CS020401000016      True     False     False
6  CS015414000103     False      True     False
7  CS029403000008      True     False     False
8  CS015804000004      True     False     False
9  CS033513000180     False      True     False


---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
from sklearn.preprocessing import StandardScaler

df_customer_sales = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().reset_index()

scaler = StandardScaler()
df_customer_sales['amount_scaled'] = scaler.fit_transform(df_customer_sales[['amount']])

print(df_customer_sales[['customer_id', 'amount_scaled']].head(10))

      customer_id  amount_scaled
0  CS001113000004      -0.459378
1  CS001114000005      -0.706390
2  CS001115000010       0.182413
3  CS001205000004      -0.205749
4  CS001205000006       0.290114
5  CS001211000025      -0.768879
6  CS001212000027      -0.771819
7  CS001212000031      -0.827691
8  CS001212000046      -0.852686
9  CS001212000070      -0.768879


TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df_customer_sales = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().reset_index()

scaler = MinMaxScaler()
df_customer_sales['amount_normalized'] = scaler.fit_transform(df_customer_sales[['amount']])

print(df_customer_sales[['customer_id', 'amount', 'amount_normalized']].head(10))

      customer_id  amount  amount_normalized
0  CS001113000004    1298           0.053354
1  CS001114000005     626           0.024157
2  CS001115000010    3044           0.129214
3  CS001205000004    1988           0.083333
4  CS001205000006    3337           0.141945
5  CS001211000025     456           0.016771
6  CS001212000027     448           0.016423
7  CS001212000031     296           0.009819
8  CS001212000046     228           0.006865
9  CS001212000070     456           0.016771


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_customer_sales = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().reset_index()

df_customer_sales['amount_log10'] = np.log10(df_customer_sales['amount'])

print(df_customer_sales[['customer_id', 'amount', 'amount_log10']].head(10))

      customer_id  amount  amount_log10
0  CS001113000004    1298      3.113275
1  CS001114000005     626      2.796574
2  CS001115000010    3044      3.483445
3  CS001205000004    1988      3.298416
4  CS001205000006    3337      3.523356
5  CS001211000025     456      2.658965
6  CS001212000027     448      2.651278
7  CS001212000031     296      2.471292
8  CS001212000046     228      2.357935
9  CS001212000070     456      2.658965


---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_customer_sales = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().reset_index()

df_customer_sales['amount_log'] = np.log(df_customer_sales['amount'])

print(df_customer_sales[['customer_id', 'amount', 'amount_log']].head(10))

      customer_id  amount  amount_log
0  CS001113000004    1298    7.168580
1  CS001114000005     626    6.439350
2  CS001115000010    3044    8.020928
3  CS001205000004    1988    7.594884
4  CS001205000006    3337    8.112827
5  CS001211000025     456    6.122493
6  CS001212000027     448    6.104793
7  CS001212000031     296    5.690359
8  CS001212000046     228    5.429346
9  CS001212000070     456    6.122493


---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
df_product['profit'] = df_product['unit_price'] - df_product['unit_cost']

print(df_product[['product_cd', 'unit_price', 'unit_cost', 'profit']].head(10))

   product_cd  unit_price  unit_cost  profit
0  P040101001       198.0      149.0    49.0
1  P040101002       218.0      164.0    54.0
2  P040101003       230.0      173.0    57.0
3  P040101004       248.0      186.0    62.0
4  P040101005       268.0      201.0    67.0
5  P040101006       298.0      224.0    74.0
6  P040101007       338.0      254.0    84.0
7  P040101008       420.0      315.0   105.0
8  P040101009       498.0      374.0   124.0
9  P040101010       580.0      435.0   145.0


---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
df_product.dropna(subset=['unit_price', 'unit_cost'], inplace=True)
df_product['profit_margin'] = (df_product['unit_price'] - df_product['unit_cost']) / df_product['unit_price']

mean_profit_margin = df_product['profit_margin'].mean()

print(f"全体平均利益率: {mean_profit_margin:.4f}")

全体平均利益率: 0.2491


---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df_product.dropna(subset=['unit_price', 'unit_cost'], inplace=True)

df_product['new_unit_price'] = np.floor(df_product['unit_cost'] / (1 - 0.30))

df_product['new_profit_margin'] = (df_product['new_unit_price'] - df_product['unit_cost']) / df_product['new_unit_price']

print(df_product[['product_cd', 'unit_price', 'unit_cost', 'new_unit_price', 'new_profit_margin']].head(10))

   product_cd  unit_price  unit_cost  new_unit_price  new_profit_margin
0  P040101001       198.0      149.0           212.0           0.297170
1  P040101002       218.0      164.0           234.0           0.299145
2  P040101003       230.0      173.0           247.0           0.299595
3  P040101004       248.0      186.0           265.0           0.298113
4  P040101005       268.0      201.0           287.0           0.299652
5  P040101006       298.0      224.0           320.0           0.300000
6  P040101007       338.0      254.0           362.0           0.298343
7  P040101008       420.0      315.0           450.0           0.300000
8  P040101009       498.0      374.0           534.0           0.299625
9  P040101010       580.0      435.0           621.0           0.299517


---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df_product.dropna(subset=['unit_price', 'unit_cost'], inplace=True)

new_prices = np.round(df_product['unit_cost'] / (1 - 0.30))

df_product['new_unit_price'] = new_prices

df_product['new_profit_margin'] = (df_product['new_unit_price'] - df_product['unit_cost']) / df_product['new_unit_price']

print(df_product[['product_cd', 'unit_price', 'unit_cost', 'new_unit_price', 'new_profit_margin']].head(10))

   product_cd  unit_price  unit_cost  new_unit_price  new_profit_margin
0  P040101001       198.0      149.0           213.0           0.300469
1  P040101002       218.0      164.0           234.0           0.299145
2  P040101003       230.0      173.0           247.0           0.299595
3  P040101004       248.0      186.0           266.0           0.300752
4  P040101005       268.0      201.0           287.0           0.299652
5  P040101006       298.0      224.0           320.0           0.300000
6  P040101007       338.0      254.0           363.0           0.300275
7  P040101008       420.0      315.0           450.0           0.300000
8  P040101009       498.0      374.0           534.0           0.299625
9  P040101010       580.0      435.0           621.0           0.299517


---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df_product.dropna(subset=['unit_price', 'unit_cost'], inplace=True)

new_prices = np.ceil(df_product['unit_cost'] / (1 - 0.30))

df_product['new_unit_price'] = new_prices

df_product['new_profit_margin'] = (df_product['new_unit_price'] - df_product['unit_cost']) / df_product['new_unit_price']

print(df_product[['product_cd', 'unit_price', 'unit_cost', 'new_unit_price', 'new_profit_margin']].head(10))

   product_cd  unit_price  unit_cost  new_unit_price  new_profit_margin
0  P040101001       198.0      149.0           213.0           0.300469
1  P040101002       218.0      164.0           235.0           0.302128
2  P040101003       230.0      173.0           248.0           0.302419
3  P040101004       248.0      186.0           266.0           0.300752
4  P040101005       268.0      201.0           288.0           0.302083
5  P040101006       298.0      224.0           320.0           0.300000
6  P040101007       338.0      254.0           363.0           0.300275
7  P040101008       420.0      315.0           451.0           0.301552
8  P040101009       498.0      374.0           535.0           0.300935
9  P040101010       580.0      435.0           622.0           0.300643


---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
tax_rate = 0.10

df_product.dropna(subset=['unit_price'], inplace=True)

tax_inclusive_prices = df_product['unit_price'] * (1 + tax_rate)
tax_inclusive_prices = tax_inclusive_prices.apply(math.floor)

df_product['tax_inclusive_price'] = tax_inclusive_prices

print(df_product[['product_cd', 'unit_price', 'tax_inclusive_price']].head(10))

   product_cd  unit_price  tax_inclusive_price
0  P040101001       198.0                  217
1  P040101002       218.0                  239
2  P040101003       230.0                  253
3  P040101004       248.0                  272
4  P040101005       268.0                  294
5  P040101006       298.0                  327
6  P040101007       338.0                  371
7  P040101008       420.0                  462
8  P040101009       498.0                  547
9  P040101010       580.0                  638


---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
df_merged = pd.merge(df_receipt, df_product, on='product_cd', how='inner')

category_07_sales = df_merged[df_merged['category_major_cd'] == '07']['amount'].sum()

customer_sales = df_merged.groupby('customer_id')['amount'].sum().reset_index()

customers_with_category_07_sales = customer_sales[customer_sales['customer_id'].isin(df_merged[df_merged['category_major_cd'] == '07']['customer_id'])]

customers_with_category_07_sales['ratio'] = customers_with_category_07_sales['amount'] / category_07_sales

print(customers_with_category_07_sales.head(10))

       customer_id  amount     ratio
0   CS001113000004    1298  0.000069
1   CS001114000005     626  0.000033
2   CS001115000010    3044  0.000161
3   CS001205000004    1988  0.000105
4   CS001205000006    3337  0.000177
6   CS001212000027     448  0.000024
7   CS001212000031     296  0.000016
8   CS001212000046     228  0.000012
9   CS001212000070     456  0.000024
11  CS001213000018     243  0.000013


<ipython-input-14-c09309c57a30>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_with_category_07_sales['ratio'] = customers_with_category_07_sales['amount'] / category_07_sales


---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id').drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'), format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')
df_tmp['elapsed_date'] = df_tmp['sales_ymd'] - df_tmp['application_date']

print(df_tmp.head(10))

       customer_id  sales_ymd application_date elapsed_date
0   CS006214000001 2018-11-03       2015-02-01    1371 days
1   CS006214000001 2017-05-09       2015-02-01     828 days
2   CS006214000001 2017-06-08       2015-02-01     858 days
4   CS006214000001 2018-10-28       2015-02-01    1365 days
7   CS006214000001 2019-09-08       2015-02-01    1680 days
8   CS006214000001 2018-01-31       2015-02-01    1095 days
9   CS006214000001 2017-07-05       2015-02-01     885 days
10  CS006214000001 2018-11-10       2015-02-01    1378 days
12  CS006214000001 2019-04-10       2015-02-01    1529 days
15  CS006214000001 2019-06-01       2015-02-01    1581 days


---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id').drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'), format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['elapsed_months'] = ((df_tmp['sales_ymd'].dt.year - df_tmp['application_date'].dt.year) * 12 +
                            (df_tmp['sales_ymd'].dt.month - df_tmp['application_date'].dt.month))

print(df_tmp.head(10))

       customer_id  sales_ymd application_date  elapsed_months
0   CS006214000001 2018-11-03       2015-02-01              45
1   CS006214000001 2017-05-09       2015-02-01              27
2   CS006214000001 2017-06-08       2015-02-01              28
4   CS006214000001 2018-10-28       2015-02-01              44
7   CS006214000001 2019-09-08       2015-02-01              55
8   CS006214000001 2018-01-31       2015-02-01              35
9   CS006214000001 2017-07-05       2015-02-01              29
10  CS006214000001 2018-11-10       2015-02-01              45
12  CS006214000001 2019-04-10       2015-02-01              50
15  CS006214000001 2019-06-01       2015-02-01              52


---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id').drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'), format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

def calculate_elapsed_years(row):
    sales_date = row['sales_ymd']
    app_date = row['application_date']
    years_diff = (sales_date.year - app_date.year) - ((sales_date.month, sales_date.day) < (app_date.month, app_date.day))
    return years_diff

df_tmp['elapsed_years'] = df_tmp.apply(calculate_elapsed_years, axis=1)

print(df_tmp.head(10))

       customer_id  sales_ymd application_date  elapsed_years
0   CS006214000001 2018-11-03       2015-02-01              3
1   CS006214000001 2017-05-09       2015-02-01              2
2   CS006214000001 2017-06-08       2015-02-01              2
4   CS006214000001 2018-10-28       2015-02-01              3
7   CS006214000001 2019-09-08       2015-02-01              4
8   CS006214000001 2018-01-31       2015-02-01              2
9   CS006214000001 2017-07-05       2015-02-01              2
10  CS006214000001 2018-11-10       2015-02-01              3
12  CS006214000001 2019-04-10       2015-02-01              4
15  CS006214000001 2019-06-01       2015-02-01              4


---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
from datetime import datetime

df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'), format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

def calculate_elapsed_seconds(row):
    sales_date = row['sales_ymd']
    app_date = row['application_date']
    sales_epoch = sales_date.timestamp()
    app_epoch = app_date.timestamp()
    elapsed_seconds = sales_epoch - app_epoch
    return calculate_elapsed_seconds

df_tmp['elapsed_seconds'] = df_tmp.apply(calculate_elapsed_seconds, axis=1)

print(df_tmp[['customer_id', 'sales_ymd', 'application_date', 'elapsed_seconds']].head(10))

      customer_id  sales_ymd application_date  \
0  CS006214000001 2018-11-03       2015-02-01   
1  CS006214000001 2017-05-09       2015-02-01   
2  CS006214000001 2017-06-08       2015-02-01   
3  CS006214000001 2017-06-08       2015-02-01   
4  CS006214000001 2018-10-28       2015-02-01   
5  CS006214000001 2018-10-28       2015-02-01   
6  CS006214000001 2017-05-09       2015-02-01   
7  CS006214000001 2019-09-08       2015-02-01   
8  CS006214000001 2018-01-31       2015-02-01   
9  CS006214000001 2017-07-05       2015-02-01   

                                     elapsed_seconds  
0  <function calculate_elapsed_seconds at 0x7bc6d...  
1  <function calculate_elapsed_seconds at 0x7bc6d...  
2  <function calculate_elapsed_seconds at 0x7bc6d...  
3  <function calculate_elapsed_seconds at 0x7bc6d...  
4  <function calculate_elapsed_seconds at 0x7bc6d...  
5  <function calculate_elapsed_seconds at 0x7bc6d...  
6  <function calculate_elapsed_seconds at 0x7bc6d...  
7  <function calcula

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'])

def calculate_days_to_monday(sales_date):
    monday = sales_date - timedelta(days=sales_date.weekday())
    days_to_monday = (sales_date - monday).days
    return monday, days_to_monday

df_tmp['monday_date'], df_tmp['days_to_monday'] = zip(*df_tmp['sales_ymd'].apply(calculate_days_to_monday))

print(df_tmp.head(10))

NameError: name 'timedelta' is not defined

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'])

def calculate_days_to_monday(sales_date):
    monday = sales_date - pd.to_timedelta(sales_date.weekday(), unit='D')  # timedelta を pd.to_timedelta で修正
    days_to_monday = (sales_date - monday).days
    return monday, days_to_monday

df_tmp['monday_date'], df_tmp['days_to_monday'] = zip(*df_tmp['sales_ymd'].apply(calculate_days_to_monday))

print(df_tmp.head(10))

      customer_id  sales_ymd monday_date  days_to_monday
0  CS006214000001 2018-11-03  2018-10-29               5
1  CS008415000097 2018-11-18  2018-11-12               6
2  CS028414000014 2017-07-12  2017-07-10               2
3  ZZ000000000000 2019-02-05  2019-02-04               1
4  CS025415000050 2018-08-21  2018-08-20               1
5  CS003515000195 2019-06-05  2019-06-03               2
6  CS024514000042 2018-12-05  2018-12-03               2
7  CS040415000178 2019-09-22  2019-09-16               6
8  ZZ000000000000 2017-05-04  2017-05-01               3
9  CS027514000015 2019-10-10  2019-10-07               3


---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [2]:
pip install scikit-learn

In [4]:
df_, stratified_sample = train_test_split(df_customer, test_size=0.1, stratify=df_customer['gender_cd'])

gender_count = stratified_sample['gender_cd'].value_counts()

print(gender_count)

gender_cd
1    1793
0     298
9     107
Name: count, dtype: int64


---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [5]:
customer_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

customer_sales['log_amount'] = np.log(customer_sales['amount'])

mean_log_amount = customer_sales['log_amount'].mean()
std_log_amount = customer_sales['log_amount'].std()

outliers = customer_sales[customer_sales['log_amount'] > mean_log_amount + 3 * std_log_amount]

print(outliers.head(10))

         customer_id    amount  log_amount
8306  ZZ000000000000  12395003   16.332804


---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [6]:
df_receipt = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

customer_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

Q1 = customer_sales['amount'].quantile(0.25)
Q3 = customer_sales['amount'].quantile(0.75)
IQR = Q3 - Q1

outliers = customer_sales[(customer_sales['amount'] < (Q1 - 1.5 * IQR)) | (customer_sales['amount'] > (Q3 + 1.5 * IQR))]

print(outliers.head(10))

         customer_id  amount
98    CS001414000048    8584
332   CS001605000009   18925
549   CS002415000594    9568
1180  CS004414000181    9584
1558  CS005415000137    8734
1733  CS006414000001    9156
1736  CS006414000029    9179
1752  CS006415000105   10042
1755  CS006415000147   12723
1757  CS006415000157   10648


---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [7]:
missing_values = df_product.isnull().sum()

print(missing_values)

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64


---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [10]:
missing_values = df_product.isnull().sum()
print("各項目の欠損数:\n", missing_values)

initial_count = df_product.shape[0]
print("削除前のレコード数:", initial_count)

missing_records_count = df_product.isnull().any(axis=1).sum()
print("欠損が発生しているレコードの数:", missing_records_count)

final_count = df_product_cleaned.shape[0]
print("削除後のレコード数:", final_count)

assert final_count == initial_count - missing_records_count
print("欠損数に応じてレコード数が減少していることを確認しました。")

各項目の欠損数:
 product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64
削除前のレコード数: 10030
欠損が発生しているレコードの数: 7
削除後のレコード数: 10023
欠損数に応じてレコード数が減少していることを確認しました。


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
>
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
>
> ファイル出力先のパスは以下のようにすること
>
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
>
> ファイル出力先のパスは以下のようにすること。
>
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
>
> ファイル出力先のパスは以下のようにすること。
>
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
>
> ファイル出力先のパスは以下のようにすること
>
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！